In [7]:
import sys
!{sys.executable} -m pip install pandas tqdm ggplot seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [13]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import re
import json
import logging
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Any
from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, filtfilt, find_peaks, freqz
import glob
import warnings
from collections import defaultdict
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                   handlers=[logging.FileHandler("smartfallmm_analysis.log"), logging.StreamHandler()])
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
np.random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Current working directory: {os.getcwd()}")

CONFIG = {
   "DATA_DIR": "data/smartfallmm/young/accelerometer/watch",
   "MODEL_PATH": "watchTousifKd.tflite",
   "OUTPUT_DIR": "accelerometer_analysis_results",
   "FILTER_CUTOFF": 7.5,
   "SAMPLING_RATE": 25.0,
   "FILTER_ORDER": 4,
   "WINDOW_SIZE_SEC": 4.0,
   "STRIDE_SEC": 0.5,
   "FIXED_COUNT": 128,
   "USE_SMV": True,
   "PEAK_HEIGHT": 1.4,
   "PEAK_DISTANCE": 50,
   "NON_FALL_PEAK_HEIGHT": 1.2,
   "NON_FALL_PEAK_DISTANCE": 100,
   "THRESHOLD": 0.5,
   "MAX_PLOTS_PER_CATEGORY": 3,
   "SAVE_INDIVIDUAL_PLOTS": True,
   "USE_SIGMOID": True
}

os.makedirs(CONFIG["OUTPUT_DIR"], exist_ok=True)
for dir_name in ["falls", "non_falls", "plots", "raw_logits", "sigmoid_logits"]:
   os.makedirs(os.path.join(CONFIG["OUTPUT_DIR"], dir_name), exist_ok=True)

def extract_activity_info(file_path):
   filename = os.path.basename(file_path)
   match = re.match(r'S(\d+)A(\d+)T(\d+)', filename)
   if match:
       subject_id = int(match.group(1))
       activity_id = int(match.group(2))
       trial_num = int(match.group(3))
       is_fall = activity_id >= 10
       return {"subject_id": subject_id, "activity_id": activity_id, "trial_num": trial_num, 
               "is_fall": is_fall, "filename": filename}
   else:
       logger.warning(f"Could not extract activity info from {filename}")
       return {"subject_id": 0, "activity_id": 0, "trial_num": 0, "is_fall": False, "filename": filename}

def design_butterworth_filter(cutoff, fs, order=4, filter_type='low'):
   nyquist = 0.5 * fs
   normal_cutoff = cutoff / nyquist
   b, a = butter(order, normal_cutoff, btype=filter_type, analog=False)
   return b, a

def apply_butterworth_filter(data, cutoff=7.5, fs=25.0, order=4, filter_type='low'):
   b, a = design_butterworth_filter(cutoff, fs, order, filter_type)
   filtered_data = data.copy()
   for i in range(1, data.shape[1]):
       filtered_data[:, i] = filtfilt(b, a, data[:, i])
   return filtered_data, b, a

def calculate_smv(data):
   x, y, z = data[:, 1], data[:, 2], data[:, 3]
   smv = np.sqrt(x**2 + y**2 + z**2)
   return smv

def selective_sliding_window(data, window_size, height, distance):
   sqrt_sum = np.sqrt(np.sum(data[:, 1:4]**2, axis=1))
   peaks, _ = find_peaks(sqrt_sum, height=height, distance=distance)
   windows = []
   for peak in peaks:
       start = max(0, peak - window_size // 2)
       end = min(len(data), start + window_size)
       if end - start < window_size:
           continue
       window = data[start:end].copy()
       windows.append(window)
   return windows

def normalize_window(window):
   normalized = window.copy()
   for i in range(1, window.shape[1]):
       scaler = StandardScaler()
       normalized[:, i] = scaler.fit_transform(window[:, i].reshape(-1, 1)).flatten()
   return normalized

def prepare_model_input(window, include_smv=True):
   if include_smv:
       smv = calculate_smv(window).reshape(-1, 1)
       features = np.hstack([window[:, 1:4], smv])
   else:
       features = window[:, 1:4]
   return np.expand_dims(features, axis=0)

def sigmoid(x):
   return 1 / (1 + np.exp(-x))

class AccelerometerProcessor:
   def __init__(self, config):
       self.config = config
       self.window_size = config["FIXED_COUNT"]
       self.use_smv = config["USE_SMV"]
       self.filter_cutoff = config["FILTER_CUTOFF"]
       self.sampling_rate = config["SAMPLING_RATE"]
       self.filter_order = config["FILTER_ORDER"]
       logger.info(f"Initialized AccelerometerProcessor with window_size={self.window_size}, use_smv={self.use_smv}")
   
   def load_csv(self, file_path):
       try:
           df = pd.read_csv(file_path, header=None)
           if df.shape[1] < 4:
               raise ValueError(f"CSV file must have at least 4 columns (time, x, y, z). Found {df.shape[1]} columns.")
           time_strs = df.iloc[:, 0].astype(str).values
           dt_series = pd.to_datetime(time_strs, errors='coerce')
           if dt_series.isnull().all():
               times = df.iloc[:, 0].astype(float).values
               if times[0] > 1e10:
                   times = times / 1000.0
               times = times - times[0]
           else:
               base_time = dt_series[0]
               times = np.array([(ts - base_time).total_seconds() for ts in dt_series])
           sensor_data = df.iloc[:, 1:4].values.astype(np.float32)
           return np.column_stack([times, sensor_data])
       except Exception as e:
           logger.error(f"Error loading CSV file {file_path}: {e}")
           raise
   
   def process_file(self, file_path):
       activity_info = extract_activity_info(file_path)
       try:
           raw_data = self.load_csv(file_path)
           logger.info(f"Loaded {file_path} with shape {raw_data.shape}")
           filtered_data, b, a = apply_butterworth_filter(
               raw_data, 
               cutoff=self.filter_cutoff,
               fs=self.sampling_rate,
               order=self.filter_order
           )
           logger.info(f"Applied Butterworth filter to {file_path}")
           is_fall = activity_info["is_fall"]
           if is_fall:
               height = self.config["PEAK_HEIGHT"]
               distance = self.config["PEAK_DISTANCE"]
           else:
               height = self.config["NON_FALL_PEAK_HEIGHT"]
               distance = self.config["NON_FALL_PEAK_DISTANCE"]
           windows = selective_sliding_window(
               filtered_data, 
               window_size=self.window_size,
               height=height,
               distance=distance
           )
           num_windows = len(windows)
           logger.info(f"Generated {num_windows} windows for {file_path}")
           processed_windows = []
           for i, window in enumerate(windows):
               smv = calculate_smv(window)
               window_with_smv = np.column_stack([window, smv])
               normalized_window = normalize_window(window_with_smv)
               model_input = prepare_model_input(normalized_window, include_smv=self.use_smv)
               window_data = {
                   "window_idx": i,
                   "raw_window": window,
                   "window_with_smv": window_with_smv,
                   "normalized_window": normalized_window,
                   "model_input": model_input
               }
               processed_windows.append(window_data)
           return {
               "file_path": file_path,
               "activity_info": activity_info,
               "raw_data": raw_data,
               "filtered_data": filtered_data,
               "filter_coefficients": {"b": b, "a": a},
               "windows": processed_windows,
               "success": True
           }
       except Exception as e:
           logger.error(f"Error processing {file_path}: {e}")
           return {
               "file_path": file_path,
               "activity_info": activity_info,
               "success": False,
               "error": str(e)
           }

class TFLiteInferenceEngine:
   def __init__(self, model_path, use_sigmoid=True):
       self.model_path = model_path
       self.interpreter = None
       self.input_details = None
       self.output_details = None
       self.threshold = CONFIG["THRESHOLD"]
       self.use_sigmoid = use_sigmoid
       self.load_model()
   
   def load_model(self):
       try:
           logger.info(f"Loading TFLite model: {self.model_path}")
           self.interpreter = tf.lite.Interpreter(model_path=self.model_path)
           self.interpreter.allocate_tensors()
           self.input_details = self.interpreter.get_input_details()
           self.output_details = self.interpreter.get_output_details()
           input_shape = self.input_details[0]['shape']
           logger.info(f"Model loaded successfully")
           logger.info(f"Input shape: {input_shape}")
           logger.info(f"Input type: {self.input_details[0]['dtype']}")
       except Exception as e:
           logger.error(f"Error loading model: {e}")
           raise
   
   def run_inference(self, input_data):
       if self.interpreter is None:
           raise ValueError("Model not loaded. Call load_model() first.")
       expected_shape = self.input_details[0]['shape']
       actual_shape = input_data.shape
       if actual_shape != tuple(expected_shape):
           logger.warning(f"Input shape mismatch. Expected {expected_shape}, got {actual_shape}")
           try:
               input_data = input_data.reshape(expected_shape)
               logger.info(f"Reshaped input to {input_data.shape}")
           except:
               logger.error(f"Cannot reshape input to match required shape {expected_shape}")
               raise ValueError(f"Input shape {actual_shape} cannot be reshaped to {expected_shape}")
       self.interpreter.set_tensor(self.input_details[0]['index'], input_data.astype(np.float32))
       self.interpreter.invoke()
       raw_output = self.interpreter.get_tensor(self.output_details[0]['index'])
       if self.use_sigmoid:
           probs = sigmoid(raw_output)
           prediction = (probs > self.threshold).astype(int)
           return {
               "raw_logits": raw_output,
               "probabilities": probs,
               "predictions": prediction
           }
       else:
           prediction = (raw_output > 0).astype(int)
           return {
               "raw_logits": raw_output,
               "probabilities": raw_output,
               "predictions": prediction
           }

def plot_filter_response(b, a, fs, cutoff, save_path=None):
   w, h = freqz(b, a, worN=8000)
   plt.figure(figsize=(10, 6))
   plt.plot(0.5*fs*w/np.pi, np.abs(h), 'b')
   plt.title('Butterworth Filter Frequency Response')
   plt.xlabel('Frequency (Hz)')
   plt.ylabel('Gain')
   plt.grid(True)
   plt.axvline(cutoff, color='red', linestyle='--')
   plt.text(cutoff+0.5, 0.5, f'Cutoff: {cutoff} Hz', color='red')
   plt.xlim(0, fs/2)
   plt.ylim(0, 1.1)
   if save_path:
       plt.savefig(save_path, dpi=300, bbox_inches='tight')
       plt.close()
   else:
       plt.show()

def plot_raw_vs_filtered(raw_data, filtered_data, title="Raw vs Filtered Accelerometer Data", save_path=None):
   plt.figure(figsize=(15, 12))
   time = raw_data[:, 0] - raw_data[0, 0]
   plt.subplot(3, 1, 1)
   plt.plot(time, raw_data[:, 1], 'r-', alpha=0.5, label='Raw X')
   plt.plot(time, filtered_data[:, 1], 'b-', label='Filtered X')
   plt.title(f"{title} - X Component")
   plt.xlabel('Time (s)')
   plt.ylabel('Acceleration')
   plt.grid(True)
   plt.legend()
   plt.subplot(3, 1, 2)
   plt.plot(time, raw_data[:, 2], 'r-', alpha=0.5, label='Raw Y')
   plt.plot(time, filtered_data[:, 2], 'b-', label='Filtered Y')
   plt.title(f"{title} - Y Component")
   plt.xlabel('Time (s)')
   plt.ylabel('Acceleration')
   plt.grid(True)
   plt.legend()
   plt.subplot(3, 1, 3)
   plt.plot(time, raw_data[:, 3], 'r-', alpha=0.5, label='Raw Z')
   plt.plot(time, filtered_data[:, 3], 'b-', label='Filtered Z')
   plt.title(f"{title} - Z Component")
   plt.xlabel('Time (s)')
   plt.ylabel('Acceleration')
   plt.grid(True)
   plt.legend()
   plt.tight_layout()
   if save_path:
       plt.savefig(save_path, dpi=300, bbox_inches='tight')
       plt.close()
   else:
       plt.show()

def plot_logits_and_probabilities(logits, probabilities, predictions, use_sigmoid=True, save_path=None):
   plt.figure(figsize=(15, 10))
   logits_values = [float(x) if isinstance(x, (np.ndarray, np.number)) else x for x in logits]
   prob_values = [float(x) if isinstance(x, (np.ndarray, np.number)) else x for x in probabilities]
   pred_values = [int(x) if isinstance(x, (np.ndarray, np.number)) else x for x in predictions]
   plt.subplot(2, 2, 1)
   colors = ['red' if p == 1 else 'blue' for p in pred_values]
   plt.scatter(range(len(logits_values)), logits_values, c=colors)
   plt.axhline(y=0, color='black', linestyle='--')
   plt.title('Raw Logits')
   plt.xlabel('Window Index')
   plt.ylabel('Logit Value')
   plt.grid(True)
   plt.subplot(2, 2, 2)
   plt.scatter(range(len(prob_values)), prob_values, c=colors)
   plt.axhline(y=CONFIG["THRESHOLD"] if use_sigmoid else 0, color='black', linestyle='--')
   plt.title('Output Values')
   plt.xlabel('Window Index')
   plt.ylabel('Value')
   if use_sigmoid:
       plt.ylim(0, 1)
   plt.grid(True)
   plt.subplot(2, 2, 3)
   if use_sigmoid:
       x = np.linspace(-5, 5, 1000)
       y = sigmoid(x)
       plt.plot(x, y, 'g-', label='Sigmoid Function')
       plt.axhline(y=CONFIG["THRESHOLD"], color='black', linestyle='--', label='Threshold')
       for i, (logit, prob) in enumerate(zip(logits_values, prob_values)):
           color = 'red' if pred_values[i] == 1 else 'blue'
           plt.scatter(logit, prob, c=color)
       plt.title('Sigmoid Transformation')
       plt.xlabel('Logit')
       plt.ylabel('Probability')
   else:
       plt.plot(logits_values, pred_values, 'g-', label='Threshold Function')
       plt.axhline(y=0.5, color='black', linestyle='--', label='Threshold')
       plt.scatter(logits_values, pred_values, c=colors)
       plt.title('Threshold Function')
       plt.xlabel('Logit')
       plt.ylabel('Prediction')
   plt.legend()
   plt.grid(True)
   plt.subplot(2, 2, 4)
   plt.hist(prob_values, bins=20, color='green', alpha=0.7)
   plt.axvline(x=CONFIG["THRESHOLD"] if use_sigmoid else 0, color='black', linestyle='--', label='Threshold')
   plt.title('Output Distribution')
   plt.xlabel('Value')
   plt.ylabel('Count')
   plt.legend()
   plt.grid(True)
   plt.tight_layout()
   if save_path:
       plt.savefig(save_path, dpi=300, bbox_inches='tight')
       plt.close()
   else:
       plt.show()

def plot_confusion_matrix(confusion_mat, classes=['Non-Fall', 'Fall'], save_path=None):
   plt.figure(figsize=(10, 8))
   sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
   plt.title('Confusion Matrix')
   plt.ylabel('True Label')
   plt.xlabel('Predicted Label')
   if save_path:
       plt.savefig(save_path, dpi=300, bbox_inches='tight')
       plt.close()
   else:
       plt.show()

def plot_metrics_comparison(sigmoid_metrics, raw_metrics, save_path=None):
   plt.figure(figsize=(15, 10))
   metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']
   sigmoid_values = [
       sigmoid_metrics['overall']['accuracy'],
       sigmoid_metrics['overall']['precision'],
       sigmoid_metrics['overall']['recall'],
       sigmoid_metrics['overall']['f1_score'],
       sigmoid_metrics['overall']['auc']
   ]
   raw_values = [
       raw_metrics['overall']['accuracy'],
       raw_metrics['overall']['precision'],
       raw_metrics['overall']['recall'],
       raw_metrics['overall']['f1_score'],
       raw_metrics['overall']['auc']
   ]
   x = np.arange(len(metric_names))
   width = 0.35
   plt.bar(x - width/2, sigmoid_values, width, label='With Sigmoid')
   plt.bar(x + width/2, raw_values, width, label='Raw Logits')
   plt.axhline(y=0.5, color='black', linestyle='--')
   plt.ylabel('Score')
   plt.title('Performance Comparison: Sigmoid vs Raw Logits')
   plt.xticks(x, metric_names)
   plt.ylim(0, 1.1)
   plt.legend()
   plt.grid(True, axis='y')
   for i, (sv, rv) in enumerate(zip(sigmoid_values, raw_values)):
       plt.text(i - width/2, sv + 0.02, f"{sv:.2f}", ha='center')
       plt.text(i + width/2, rv + 0.02, f"{rv:.2f}", ha='center')
   if save_path:
       plt.savefig(save_path, dpi=300, bbox_inches='tight')
       plt.close()
   else:
       plt.show()

def analyze_files_with_engine(use_sigmoid=True):
   processor = AccelerometerProcessor(CONFIG)
   engine = TFLiteInferenceEngine(CONFIG["MODEL_PATH"], use_sigmoid=use_sigmoid)
   data_dir = CONFIG["DATA_DIR"]
   
   if not os.path.exists(data_dir):
       logger.error(f"Directory not found: {data_dir}")
       return None
   
   csv_files = sorted(glob.glob(os.path.join(data_dir, "*.csv")))
   if not csv_files:
       logger.error(f"No CSV files found in {data_dir}")
       return None
   
   logger.info(f"Found {len(csv_files)} CSV files")
   
   output_subdir = "sigmoid_logits" if use_sigmoid else "raw_logits"
   output_dir = os.path.join(CONFIG["OUTPUT_DIR"], output_subdir)
   os.makedirs(output_dir, exist_ok=True)
   
   all_results = []
   fall_logits = []
   non_fall_logits = []
   fall_probs = []
   non_fall_probs = []
   true_labels = []
   pred_labels = []
   metrics_by_activity = defaultdict(list)
   
   for file_idx, file_path in enumerate(tqdm(csv_files, desc=f"Processing files ({'sigmoid' if use_sigmoid else 'raw'})")):
       result = processor.process_file(file_path)
       
       if not result["success"]:
           logger.warning(f"Failed to process {file_path}: {result.get('error', 'Unknown error')}")
           continue
       
       activity_info = result["activity_info"]
       activity_id = activity_info["activity_id"]
       is_fall = activity_info["is_fall"]
       
       if file_idx < CONFIG["MAX_PLOTS_PER_CATEGORY"] and CONFIG["SAVE_INDIVIDUAL_PLOTS"]:
           filter_plot_path = os.path.join(output_dir, f"{activity_info['filename']}_filter_response.png")
           plot_filter_response(
               result["filter_coefficients"]["b"],
               result["filter_coefficients"]["a"],
               CONFIG["SAMPLING_RATE"],
               CONFIG["FILTER_CUTOFF"],
               save_path=filter_plot_path
           )
           
           filtered_plot_path = os.path.join(output_dir, f"{activity_info['filename']}_raw_vs_filtered.png")
           plot_raw_vs_filtered(
               result["raw_data"],
               result["filtered_data"],
               title=f"Activity {activity_id} - {'Fall' if is_fall else 'Non-Fall'}",
               save_path=filtered_plot_path
           )
       
       window_logits = []
       window_probs = []
       window_preds = []
       
       for window_idx, window in enumerate(result["windows"]):
           inference_result = engine.run_inference(window["model_input"])
           window["inference"] = inference_result
           
           logit = float(inference_result["raw_logits"])
           prob = float(inference_result["probabilities"])
           pred = int(inference_result["predictions"])
           
           window_logits.append(logit)
           window_probs.append(prob)
           window_preds.append(pred)
           
           if is_fall:
               fall_logits.append(logit)
               fall_probs.append(prob)
           else:
               non_fall_logits.append(logit)
               non_fall_probs.append(prob)
           
       if window_preds:
           majority_pred = 1 if sum(window_preds) > len(window_preds) / 2 else 0
           avg_prob = sum(window_probs) / len(window_probs)
           
           true_labels.append(int(is_fall))
           pred_labels.append(majority_pred)
           
           metrics_by_activity[activity_id].append({
               "file_path": file_path,
               "window_count": len(window_preds),
               "true_label": int(is_fall),
               "pred_label": majority_pred,
               "avg_prob": avg_prob,
               "correct": majority_pred == int(is_fall)
           })
           
           if file_idx < CONFIG["MAX_PLOTS_PER_CATEGORY"] and CONFIG["SAVE_INDIVIDUAL_PLOTS"]:
               logits_plot_path = os.path.join(
                   output_dir, 
                   f"{activity_info['filename']}_logits_and_probs.png"
               )
               plot_logits_and_probabilities(
                   window_logits,
                   window_probs,
                   window_preds,
                   use_sigmoid=use_sigmoid,
                   save_path=logits_plot_path
               )
           
           file_result = {
               "file_path": file_path,
               "activity_info": activity_info,
               "window_count": len(window_preds),
               "fall_window_count": sum(window_preds),
               "non_fall_window_count": len(window_preds) - sum(window_preds),
               "majority_prediction": majority_pred,
               "average_probability": avg_prob,
               "correct_classification": majority_pred == int(is_fall),
               "window_logits": window_logits,
               "window_probs": window_probs,
               "window_preds": window_preds
           }
           
           all_results.append(file_result)
           
           result_path = os.path.join(output_dir, f"{activity_info['filename']}_result.json")
           with open(result_path, 'w') as f:
               simplified_result = {k: v for k, v in file_result.items() 
                                 if k not in ["window_logits", "window_probs", "window_preds"]}
               json.dump(simplified_result, f, indent=2)
   
   if all_results:
       conf_matrix = confusion_matrix(true_labels, pred_labels)
       
       plot_confusion_matrix(
           conf_matrix,
           save_path=os.path.join(output_dir, f"confusion_matrix.png")
       )
       
       activity_metrics = {}
       
       for activity_id, results in metrics_by_activity.items():
           if not results:
               continue
               
           true_labels_act = [r["true_label"] for r in results]
           pred_labels_act = [r["pred_label"] for r in results]
           
           if len(set(true_labels_act)) != 1:
               logger.warning(f"Activity {activity_id} has mixed true labels: {true_labels_act}")
               continue
               
           is_fall = true_labels_act[0] == 1
           
           accuracy = accuracy_score(true_labels_act, pred_labels_act)
           precision = precision_score(true_labels_act, pred_labels_act, zero_division=0)
           recall = recall_score(true_labels_act, pred_labels_act, zero_division=0)
           f1 = f1_score(true_labels_act, pred_labels_act, zero_division=0)
           
           try:
               avg_probs = [r["avg_prob"] for r in results]
               auc = roc_auc_score(true_labels_act, avg_probs)
           except:
               auc = 0.0
           
           activity_metrics[activity_id] = {
               "is_fall": is_fall,
               "file_count": len(results),
               "correct_count": sum(1 for r in results if r["correct"]),
               "accuracy": accuracy,
               "precision": precision,
               "recall": recall,
               "f1_score": f1,
               "auc": auc
           }
       
       fall_results = [r for r in all_results if r["activity_info"]["is_fall"]]
       non_fall_results = [r for r in all_results if not r["activity_info"]["is_fall"]]
       
       tp = sum(1 for r in fall_results if r["majority_prediction"] == 1)
       fp = sum(1 for r in non_fall_results if r["majority_prediction"] == 1)
       tn = sum(1 for r in non_fall_results if r["majority_prediction"] == 0)
       fn = sum(1 for r in fall_results if r["majority_prediction"] == 0)
       
       overall_accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
       overall_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
       overall_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
       overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
       
       fall_labels = [1] * len(fall_results)
       non_fall_labels = [0] * len(non_fall_results)
       
       all_labels = fall_labels + non_fall_labels
       all_probs = [r["average_probability"] for r in fall_results + non_fall_results]
       
       try:
           overall_auc = roc_auc_score(all_labels, all_probs)
       except:
           overall_auc = 0.0
       
       fall_accuracy = tp / len(fall_results) if fall_results else 0
       non_fall_accuracy = tn / len(non_fall_results) if non_fall_results else 0
       
       metrics = {
           "overall": {
               "accuracy": overall_accuracy,
               "precision": overall_precision,
               "recall": overall_recall,
               "f1_score": overall_f1,
               "auc": overall_auc,
               "total_files": len(all_results),
               "fall_files": len(fall_results),
               "non_fall_files": len(non_fall_results)
           },
           "fall": {
               "accuracy": fall_accuracy,
               "precision": tp / (tp + fp) if (tp + fp) > 0 else 0,
               "recall": tp / (tp + fn) if (tp + fn) > 0 else 0,
               "f1_score": 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0,
               "auc": roc_auc_score(fall_labels, [r["average_probability"] for r in fall_results]) if fall_results else 0,
               "total_files": len(fall_results)
           },
           "non_fall": {
               "accuracy": non_fall_accuracy,
               "precision": tn / (tn + fn) if (tn + fn) > 0 else 0,
               "recall": tn / (tn + fp) if (tn + fp) > 0 else 0,
               "f1_score": 2 * tn / (2 * tn + fn + fp) if (2 * tn + fn + fp) > 0 else 0,
               "auc": roc_auc_score([1-l for l in non_fall_labels], [1-r["average_probability"] for r in non_fall_results]) if non_fall_results else 0,
               "total_files": len(non_fall_results)
           },
           "by_activity": activity_metrics
       }
       
       metrics_path = os.path.join(output_dir, "metrics.json")
       with open(metrics_path, 'w') as f:
           json.dump(metrics, f, indent=2)
           
       print("\n" + "="*50)
       print(f"DATASET INFERENCE SUMMARY ({'SIGMOID' if use_sigmoid else 'RAW LOGITS'})")
       print("="*50)
       print(f"Total files processed: {len(all_results)}")
       print(f"Fall files: {len(fall_results)}")
       print(f"Non-fall files: {len(non_fall_results)}")
       print(f"Overall accuracy: {overall_accuracy:.2%}")
       print(f"Fall accuracy: {fall_accuracy:.2%}")
       print(f"Non-fall accuracy: {non_fall_accuracy:.2%}")
       print(f"Precision: {overall_precision:.2%}")
       print(f"Recall: {overall_recall:.2%}")
       print(f"F1 Score: {overall_f1:.2%}")
       print(f"AUC: {overall_auc:.2%}")
       print("="*50)
       
       return metrics
   else:
       logger.warning("No results to analyze")
       return None

def run_complete_analysis():
   print("Starting analysis with sigmoid activation...")
   sigmoid_metrics = analyze_files_with_engine(use_sigmoid=True)
   
   print("\nStarting analysis with raw logits (no sigmoid)...")
   raw_metrics = analyze_files_with_engine(use_sigmoid=False)
   
   if sigmoid_metrics and raw_metrics:
       plot_metrics_comparison(
           sigmoid_metrics, 
           raw_metrics,
           save_path=os.path.join(CONFIG["OUTPUT_DIR"], "sigmoid_vs_raw_comparison.png")
       )
       
       comparison = {
           "sigmoid": sigmoid_metrics["overall"],
           "raw_logits": raw_metrics["overall"],
           "difference": {
               "accuracy": sigmoid_metrics["overall"]["accuracy"] - raw_metrics["overall"]["accuracy"],
               "precision": sigmoid_metrics["overall"]["precision"] - raw_metrics["overall"]["precision"],
               "recall": sigmoid_metrics["overall"]["recall"] - raw_metrics["overall"]["recall"],
               "f1_score": sigmoid_metrics["overall"]["f1_score"] - raw_metrics["overall"]["f1_score"],
               "auc": sigmoid_metrics["overall"]["auc"] - raw_metrics["overall"]["auc"]
           }
       }
       
       with open(os.path.join(CONFIG["OUTPUT_DIR"], "comparison_metrics.json"), 'w') as f:
           json.dump(comparison, f, indent=2)
       
       print("\n" + "="*50)
       print("SIGMOID VS RAW LOGITS COMPARISON")
       print("="*50)
       print(f"Accuracy: Sigmoid: {comparison['sigmoid']['accuracy']:.2%}, Raw: {comparison['raw_logits']['accuracy']:.2%}, Diff: {comparison['difference']['accuracy']:.2%}")
       print(f"Precision: Sigmoid: {comparison['sigmoid']['precision']:.2%}, Raw: {comparison['raw_logits']['precision']:.2%}, Diff: {comparison['difference']['precision']:.2%}")
       print(f"Recall: Sigmoid: {comparison['sigmoid']['recall']:.2%}, Raw: {comparison['raw_logits']['recall']:.2%}, Diff: {comparison['difference']['recall']:.2%}")
       print(f"F1 Score: Sigmoid: {comparison['sigmoid']['f1_score']:.2%}, Raw: {comparison['raw_logits']['f1_score']:.2%}, Diff: {comparison['difference']['f1_score']:.2%}")
       print(f"AUC: Sigmoid: {comparison['sigmoid']['auc']:.2%}, Raw: {comparison['raw_logits']['auc']:.2%}, Diff: {comparison['difference']['auc']:.2%}")
       print("="*50)
       
       return comparison
   else:
       print("Could not perform comparison as one or both analyses failed.")
       return None

print("Starting complete analysis...")
comparison = run_complete_analysis()

2025-04-14 02:58:54,777 - __main__ - INFO - Initialized AccelerometerProcessor with window_size=128, use_smv=True
2025-04-14 02:58:54,779 - __main__ - INFO - Loading TFLite model: watchTousifKd.tflite
2025-04-14 02:58:54,781 - __main__ - INFO - Model loaded successfully
2025-04-14 02:58:54,784 - __main__ - INFO - Input shape: [  1 128   4]
2025-04-14 02:58:54,784 - __main__ - INFO - Input type: <class 'numpy.float32'>
2025-04-14 02:58:54,792 - __main__ - INFO - Found 899 CSV files


TensorFlow version: 2.20.0-dev20250309
NumPy version: 2.1.3
Pandas version: 2.2.3
Current working directory: /home/abheekp/tousifWeights/Torch_tflite
Starting complete analysis...
Starting analysis with sigmoid activation...


Processing files (sigmoid):   0%|          | 0/899 [00:00<?, ?it/s]

2025-04-14 02:58:54,813 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A01T01.csv with shape (1109, 4)
2025-04-14 02:58:54,816 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A01T01.csv
2025-04-14 02:58:54,818 - __main__ - INFO - Generated 7 windows for data/smartfallmm/young/accelerometer/watch/S28A01T01.csv
2025-04-14 02:58:57,488 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A02T01.csv with shape (576, 4)
2025-04-14 02:58:57,492 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A02T01.csv
2025-04-14 02:58:57,494 - __main__ - INFO - Generated 4 windows for data/smartfallmm/young/accelerometer/watch/S28A02T01.csv
2025-04-14 02:59:00,394 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A02T02.csv with shape (388, 4)
2025-04-14 02:59:00,396 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accele

2025-04-14 02:59:03,273 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A08T01.csv
2025-04-14 02:59:03,276 - __main__ - INFO - Generated 3 windows for data/smartfallmm/young/accelerometer/watch/S28A08T01.csv
2025-04-14 02:59:03,292 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A08T03.csv with shape (411, 4)
2025-04-14 02:59:03,296 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A08T03.csv
2025-04-14 02:59:03,298 - __main__ - INFO - Generated 3 windows for data/smartfallmm/young/accelerometer/watch/S28A08T03.csv
2025-04-14 02:59:03,319 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A09T02.csv with shape (133, 4)
2025-04-14 02:59:03,322 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A09T02.csv
2025-04-14 02:59:03,324 - __main__ - INFO - Generated 1 windows for data/smartfallmm/young/accelerome

...

2025-04-14 03:00:03,173 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A02T02.csv
2025-04-14 03:00:03,175 - __main__ - INFO - Generated 22 windows for data/smartfallmm/young/accelerometer/watch/S46A02T02.csv
2025-04-14 03:00:03,239 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A02T03.csv with shape (2904, 4)
2025-04-14 03:00:03,242 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A02T03.csv
2025-04-14 03:00:03,244 - __main__ - INFO - Generated 22 windows for data/smartfallmm/young/accelerometer/watch/S46A02T03.csv
2025-04-14 03:00:03,327 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A02T04.csv with shape (2904, 4)
2025-04-14 03:00:03,331 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A02T04.csv
2025-04-14 03:00:03,333 - __main__ - INFO - Generated 22 windows for data/smartfallmm/young/accel

2025-04-14 03:00:04,756 - __main__ - INFO - Generated 22 windows for data/smartfallmm/young/accelerometer/watch/S46A06T03.csv
2025-04-14 03:00:04,818 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A06T04.csv with shape (2904, 4)
2025-04-14 03:00:04,821 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A06T04.csv
2025-04-14 03:00:04,822 - __main__ - INFO - Generated 22 windows for data/smartfallmm/young/accelerometer/watch/S46A06T04.csv
2025-04-14 03:00:04,883 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A06T05.csv with shape (2904, 4)
2025-04-14 03:00:04,886 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A06T05.csv
2025-04-14 03:00:04,888 - __main__ - INFO - Generated 22 windows for data/smartfallmm/young/accelerometer/watch/S46A06T05.csv
2025-04-14 03:00:04,949 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A07T0

2025-04-14 03:00:05,615 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A10T04.csv with shape (1172, 4)
2025-04-14 03:00:05,618 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A10T04.csv
2025-04-14 03:00:05,620 - __main__ - INFO - Generated 17 windows for data/smartfallmm/young/accelerometer/watch/S46A10T04.csv
2025-04-14 03:00:05,664 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A10T05.csv with shape (1176, 4)
2025-04-14 03:00:05,666 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A10T05.csv
2025-04-14 03:00:05,668 - __main__ - INFO - Generated 15 windows for data/smartfallmm/young/accelerometer/watch/S46A10T05.csv
2025-04-14 03:00:05,715 - __main__ - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A11T01.csv with shape (1586, 4)
2025-04-14 03:00:05,719 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/ac

2025-04-14 03:00:06,803 - __main__ - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A14T05.csv
2025-04-14 03:00:06,804 - __main__ - INFO - Generated 8 windows for data/smartfallmm/young/accelerometer/watch/S46A14T05.csv



DATASET INFERENCE SUMMARY (RAW LOGITS)
Total files processed: 735
Fall files: 305
Non-fall files: 430
Overall accuracy: 41.50%
Fall accuracy: 100.00%
Non-fall accuracy: 0.00%
Precision: 41.50%
Recall: 100.00%
F1 Score: 58.65%
AUC: 51.66%

SIGMOID VS RAW LOGITS COMPARISON
Accuracy: Sigmoid: 41.50%, Raw: 41.50%, Diff: 0.00%
Precision: Sigmoid: 41.50%, Raw: 41.50%, Diff: 0.00%
Recall: Sigmoid: 100.00%, Raw: 100.00%, Diff: 0.00%
F1 Score: Sigmoid: 58.65%, Raw: 58.65%, Diff: 0.00%
AUC: Sigmoid: 51.62%, Raw: 51.66%, Diff: -0.04%


In [9]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm.notebook import tqdm
import glob
import json
from collections import defaultdict
import logging
import re

# Configure logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger('PeakAlignedProcessor')

class PeakAlignedProcessor:
    """
    Processes accelerometer data using peak-aligned window extraction matching
    the Java implementation for SmartFall detection with moving average
    aggregation for file-level predictions.
    """
    
    def __init__(self, config):
        """
        Initialize the processor with configuration settings
        """
        self.config = config
        self.window_size = config["WINDOW_SIZE"]
        self.cutoff_freq = config["FILTER_CUTOFF"]
        self.sampling_freq = config["SAMPLING_RATE"]
        self.filter_order = config["FILTER_ORDER"]
        self.threshold = config["THRESHOLD"]
        self.output_dir = config["OUTPUT_DIR"]
        
        # Peak detection parameters
        self.fall_height = config["FALL_PEAK_HEIGHT"]
        self.fall_distance = config["FALL_PEAK_DISTANCE"]
        self.nonfall_height = config["NONFALL_PEAK_HEIGHT"]
        self.nonfall_distance = config["NONFALL_PEAK_DISTANCE"]
        
        # Moving average parameters
        self.ma_window_size = config.get("MA_WINDOW_SIZE", 5)  # Number of windows to average
        self.decision_threshold = config.get("DECISION_THRESHOLD", 0.5)  # Threshold for file prediction
        
        # Butterworth filter coefficients
        nyquist = self.sampling_freq / 2.0
        normalized_cutoff = self.cutoff_freq / nyquist
        self.b, self.a = butter(self.filter_order, normalized_cutoff, btype='low')
        
        # Initialize TFLite interpreter
        self.interpreter = None
        
    def load_model(self, model_path=None):
        """
        Loads a TFLite model from the specified path
        """
        if model_path is None:
            model_path = self.config["MODEL_PATH"]
            
        try:
            self.interpreter = tf.lite.Interpreter(model_path=model_path)
            self.interpreter.allocate_tensors()
            self.input_details = self.interpreter.get_input_details()
            self.output_details = self.interpreter.get_output_details()
            logger.info(f"Model loaded successfully from {model_path}")
            logger.info(f"Input shape: {self.input_details[0]['shape']}")
            logger.info(f"Input type: {self.input_details[0]['dtype']}")
            return True
        except Exception as e:
            logger.error(f"Error loading model: {e}")
            return False
    
    def apply_butterworth_filter(self, data):
        """
        Apply Butterworth filter to accelerometer data
        """
        filtered_data = np.copy(data)
        for i in range(data.shape[1]):
            filtered_data[:, i] = filtfilt(self.b, self.a, data[:, i])
        return filtered_data
    
    def calculate_smv(self, data):
        """
        Calculate signal magnitude vector (SMV) for accelerometer data
        """
        return np.sqrt(np.sum(data**2, axis=1))
    
    def extract_peak_aligned_windows(self, data, is_fall=False):
        """
        Extract windows aligned with peaks in the SMV
        
        Args:
            data: Filtered accelerometer data (N, 3)
            is_fall: Whether to use fall or non-fall peak detection parameters
            
        Returns:
            List of windows of shape (window_size, 3)
        """
        # Calculate SMV
        smv = self.calculate_smv(data)
        
        # Use appropriate peak detection parameters
        if is_fall:
            height = self.fall_height
            distance = self.fall_distance
        else:
            height = self.nonfall_height
            distance = self.nonfall_distance
        
        # Find peaks in the SMV
        peaks, _ = find_peaks(smv, height=height, distance=distance)
        
        # Extract windows centered on peaks
        windows = []
        valid_peaks = []
        
        for peak in peaks:
            # Calculate window boundaries
            start = max(0, peak - self.window_size // 2)
            end = min(len(data), start + self.window_size)
            
            # Skip if we can't extract a full window
            if end - start < self.window_size:
                continue
            
            # Extract window
            window = data[start:end]
            windows.append(window)
            valid_peaks.append(peak)
        
        return windows, np.array(valid_peaks)
    
    def flatten_input_to_3d(self, samples):
        """
        Converts 2D samples array to 3D format with SMV as fourth feature,
        calculated exactly as in the Java implementation
        """
        size = samples.shape[0]
        features = samples.shape[1] + 1  # x, y, z + smv
        multi_dim_array = np.zeros((1, size, features))
        
        # Calculate mean for each axis (across time/window dimension)
        mean_x = np.mean(samples[:, 0])
        mean_y = np.mean(samples[:, 1])
        mean_z = np.mean(samples[:, 2])
        
        # Process each sample
        for i in range(size):
            # Copy the x, y, z values
            multi_dim_array[0, i, 0] = samples[i, 0]
            multi_dim_array[0, i, 1] = samples[i, 1]
            multi_dim_array[0, i, 2] = samples[i, 2]
            
            # Calculate zero-mean values
            zero_mean_x = samples[i, 0] - mean_x
            zero_mean_y = samples[i, 1] - mean_y
            zero_mean_z = samples[i, 2] - mean_z
            
            # Calculate sum of squares
            sum_squared = zero_mean_x**2 + zero_mean_y**2 + zero_mean_z**2
            
            # Calculate SMV as sqrt(sum_squared)
            multi_dim_array[0, i, 3] = np.sqrt(sum_squared)
            
        return multi_dim_array
    
    def check_activity(self, window):
        """
        Implements the activity check from the Java code
        Returns True if window should be processed, False otherwise
        """
        for i in range(window.shape[0]):
            num_high_values = 0
            for j in range(3):  # For x, y, z
                if window[i, j] > 8 or window[i, j] < -8:
                    num_high_values += 1
            
            if num_high_values >= 2:
                return True
        
        return False
    
    def extract_activity_info(self, file_path):
        """
        Extract activity info from filename (expects SmartFallMM format: S01A01T01.csv)
        """
        filename = os.path.basename(file_path)
        match = re.match(r'S(\d+)A(\d+)T(\d+)', filename)
        
        if match:
            subject_id = int(match.group(1))
            activity_id = int(match.group(2))
            trial_num = int(match.group(3))
            is_fall = activity_id >= 10  # Assuming activities 10+ are falls
            
            return {
                "subject_id": subject_id, 
                "activity_id": activity_id, 
                "trial_num": trial_num, 
                "is_fall": is_fall, 
                "filename": filename
            }
        else:
            logger.warning(f"Could not extract activity info from {filename}")
            return {
                "subject_id": 0, 
                "activity_id": 0, 
                "trial_num": 0, 
                "is_fall": False, 
                "filename": filename
            }
    
    def load_csv(self, file_path):
        """
        Loads CSV file containing accelerometer data
        """
        try:
            # Try with csv-specific parsing first
            try:
                df = pd.read_csv(file_path, header=None)
            except:
                # Try opening as a text file and parsing manually
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                
                # Parse data from lines
                data = []
                for line in lines:
                    parts = line.strip().split(',')
                    if len(parts) >= 4:  # We need at least time + xyz
                        data.append(parts)
                
                if not data:
                    raise ValueError(f"No valid data found in file {file_path}")
                    
                df = pd.DataFrame(data)
            
            # Check if there are at least 4 columns (time, x, y, z)
            if df.shape[1] < 4:
                raise ValueError(f"CSV must have at least 4 columns, found {df.shape[1]}")
            
            # Convert columns to float
            sensor_data = np.zeros((len(df), 3))
            for i in range(3):
                sensor_data[:, i] = pd.to_numeric(df.iloc[:, i+1], errors='coerce')
            
            # Make sure data is float32
            sensor_data = sensor_data.astype(np.float32)
            
            return sensor_data
            
        except Exception as e:
            logger.error(f"Error loading CSV file {file_path}: {e}")
            raise
    
    def apply_moving_average(self, predictions, window_size=None):
        """
        Apply moving average to predictions
        
        Args:
            predictions: List of prediction values
            window_size: Size of moving average window (default: self.ma_window_size)
            
        Returns:
            List of smoothed predictions
        """
        if window_size is None:
            window_size = self.ma_window_size
        
        if len(predictions) < window_size:
            # If we have fewer predictions than window size, just use them all
            return [sum(predictions) / len(predictions)] * len(predictions)
        
        # Apply moving average
        smoothed = []
        for i in range(len(predictions)):
            start = max(0, i - window_size // 2)
            end = min(len(predictions), i + window_size // 2 + 1)
            window_vals = predictions[start:end]
            smoothed.append(sum(window_vals) / len(window_vals))
        
        return smoothed
    
    def process_file(self, file_path):
        """
        Process a single CSV file and return predictions using peak-aligned window extraction
        and moving average for file-level decision
        """
        if self.interpreter is None:
            raise ValueError("Model not loaded. Call load_model() first.")
            
        activity_info = self.extract_activity_info(file_path)
        
        try:
            # Load and filter data
            raw_data = self.load_csv(file_path)
            logger.info(f"Loaded {file_path} with shape {raw_data.shape}")
            
            # Apply filtering
            filtered_data = self.apply_butterworth_filter(raw_data)
            logger.info(f"Applied Butterworth filter to {file_path}")
            
            # Extract peak-aligned windows
            is_fall = activity_info["is_fall"]
            windows, peaks = self.extract_peak_aligned_windows(filtered_data, is_fall)
            logger.info(f"Extracted {len(windows)} peak-aligned windows for {file_path}")
            
            predictions = []
            raw_outputs = []
            window_results = []
            
            for i, window in enumerate(windows):
                window_result = {"window_idx": i, "peak_idx": int(peaks[i])}
                
                # Check if window has enough activity
                has_activity = self.check_activity(window)
                window_result["has_activity"] = has_activity
                
                if not has_activity:
                    # Return random low value as in Java
                    random_val = np.random.randint(15) / 50
                    predictions.append(float(random_val))
                    raw_outputs.append([float(random_val), 0.0])
                    window_result["prediction"] = float(random_val)
                    window_result["is_random"] = True
                    window_results.append(window_result)
                    continue
                
                # Prepare input for model
                model_input = self.flatten_input_to_3d(window)
                window_result["model_input_shape"] = list(model_input.shape)
                
                # Run inference
                self.interpreter.set_tensor(self.input_details[0]['index'], model_input.astype(np.float32))
                self.interpreter.invoke()
                output = self.interpreter.get_tensor(self.output_details[0]['index'])
                
                # Convert numpy values to Python native types for JSON serialization
                raw_output_list = [float(x) for x in output[0]]
                raw_outputs.append(raw_output_list)
                predictions.append(float(output[0][0]))
                
                window_result["raw_output"] = raw_output_list
                window_result["prediction"] = float(output[0][0])
                window_result["is_fall"] = float(output[0][0]) > self.threshold
                window_result["is_random"] = False
                
                window_results.append(window_result)
            
            # Apply moving average to window predictions
            if predictions:
                smoothed_predictions = self.apply_moving_average(predictions)
                
                # Determine file-level prediction using average of all smoothed predictions
                avg_prediction = sum(smoothed_predictions) / len(smoothed_predictions)
                is_fall_file = avg_prediction > self.decision_threshold
                
                # Calculate additional metrics
                thresholded_preds = [1 if p > self.threshold else 0 for p in predictions]
                fall_windows = sum(thresholded_preds)
                
                result = {
                    "file_path": file_path,
                    "activity_info": activity_info,
                    "num_windows": len(predictions),
                    "fall_windows": fall_windows,
                    "non_fall_windows": len(predictions) - fall_windows,
                    "is_fall_file": is_fall_file,
                    "correct_classification": is_fall_file == activity_info["is_fall"],
                    "average_probability": float(avg_prediction),
                    "raw_predictions": predictions,
                    "smoothed_predictions": smoothed_predictions,
                    "thresholded_predictions": thresholded_preds,
                    "window_results": window_results,
                    "peaks": peaks.tolist(),
                }
            else:
                # If no predictions (likely no peaks found), default to non-fall
                result = {
                    "file_path": file_path,
                    "activity_info": activity_info,
                    "num_windows": 0,
                    "fall_windows": 0,
                    "non_fall_windows": 0,
                    "is_fall_file": False,
                    "correct_classification": not activity_info["is_fall"],
                    "average_probability": 0.0,
                    "raw_predictions": [],
                    "smoothed_predictions": [],
                    "thresholded_predictions": [],
                    "window_results": [],
                    "peaks": [],
                }
            
            # Keep raw and filtered data for plotting
            result["_raw_data"] = raw_data
            result["_filtered_data"] = filtered_data
            
            return result
        except Exception as e:
            logger.error(f"Error processing {file_path}: {e}")
            return {
                "file_path": file_path,
                "activity_info": activity_info,
                "success": False,
                "error": str(e)
            }
    
    def plot_raw_vs_filtered_with_peaks(self, raw_data, filtered_data, peaks, title="Data with Peaks", save_path=None):
        """
        Plot raw and filtered data with detected peaks
        """
        plt.figure(figsize=(15, 12))
        
        # Plot raw vs filtered for each axis
        for i in range(3):
            plt.subplot(4, 1, i+1)
            plt.plot(raw_data[:, i], 'r-', alpha=0.5, label=f'Raw Axis {i}')
            plt.plot(filtered_data[:, i], 'b-', label=f'Filtered Axis {i}')
            plt.title(f"{title} - Axis {i}")
            plt.grid(True)
            plt.legend()
        
        # Plot SMV with detected peaks
        plt.subplot(4, 1, 4)
        smv = self.calculate_smv(filtered_data)
        plt.plot(smv, 'g-', label='Signal Magnitude')
        if len(peaks) > 0:
            plt.plot(peaks, smv[peaks], 'ro', label='Detected Peaks')
        plt.title("Signal Magnitude with Detected Peaks")
        plt.grid(True)
        plt.legend()
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
    
    def plot_predictions_with_ma(self, raw_preds, smoothed_preds, title="Predictions with Moving Average", save_path=None):
        """
        Plot raw and smoothed predictions
        """
        plt.figure(figsize=(12, 8))
        
        # Plot raw and smoothed predictions
        plt.subplot(2, 1, 1)
        plt.plot(raw_preds, 'b-', alpha=0.6, label='Raw Predictions')
        plt.plot(smoothed_preds, 'r-', linewidth=2, label='Moving Average')
        plt.axhline(y=self.threshold, color='black', linestyle='--', label='Window Threshold')
        plt.axhline(y=self.decision_threshold, color='green', linestyle='--', label='File Decision Threshold')
        plt.title('Predictions with Moving Average')
        plt.xlabel('Window Index')
        plt.ylabel('Prediction Value')
        plt.grid(True)
        plt.legend()
        
        # Plot histogram of predictions
        plt.subplot(2, 1, 2)
        plt.hist(raw_preds, bins=20, alpha=0.5, label='Raw Predictions')
        plt.hist(smoothed_preds, bins=20, alpha=0.5, label='Smoothed Predictions')
        plt.axvline(x=self.threshold, color='black', linestyle='--', label='Window Threshold')
        plt.axvline(x=self.decision_threshold, color='green', linestyle='--', label='File Decision Threshold')
        plt.title('Prediction Distribution')
        plt.xlabel('Prediction Value')
        plt.ylabel('Count')
        plt.grid(True)
        plt.legend()
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()
        else:
            plt.show()
    
    def process_directory(self, data_dir=None, max_files=None):
        """
        Process all CSV files in a directory using peak-aligned window extraction
        and moving average for file-level decisions
        """
        if self.interpreter is None:
            raise ValueError("Model not loaded. Call load_model() first.")
        
        if data_dir is None:
            data_dir = self.config["DATA_DIR"]
        
        # Find all CSV files
        csv_files = sorted(glob.glob(os.path.join(data_dir, "**/*.csv"), recursive=True))
        
        if not csv_files:
            logger.error(f"No CSV files found in {data_dir}")
            return None
        
        if max_files:
            csv_files = csv_files[:max_files]
        
        logger.info(f"Found {len(csv_files)} CSV files, processing...")
        
        # Process each file
        results = []
        true_labels = []
        pred_labels = []
        metrics_by_activity = defaultdict(list)
        
        for file_idx, file_path in enumerate(tqdm(csv_files, desc="Processing files")):
            try:
                result = self.process_file(file_path)
                
                if not result.get('success', True):
                    logger.warning(f"Failed to process {file_path}: {result.get('error', 'Unknown error')}")
                    continue
                
                # Save detailed results for this file
                result_path = os.path.join(self.output_dir, "raw_outputs", f"{result['activity_info']['filename']}_result.json")
                with open(result_path, 'w') as f:
                    # Create a simplified version without large arrays
                    simplified_result = {k: v for k, v in result.items() 
                                    if k not in ["raw_predictions", "smoothed_predictions", "thresholded_predictions", 
                                                "window_results", "_raw_data", "_filtered_data"]}
                    json.dump(simplified_result, f, indent=2)
                
                # Generate plots for sample files
                if file_idx < self.config["MAX_PLOTS_PER_CATEGORY"]:
                    # Plot raw vs filtered with peaks
                    peaks = np.array(result["peaks"])
                    filter_plot_path = os.path.join(self.output_dir, "plots", f"{result['activity_info']['filename']}_data_with_peaks.png")
                    self.plot_raw_vs_filtered_with_peaks(
                        result["_raw_data"],
                        result["_filtered_data"],
                        peaks,
                        title=f"Activity {result['activity_info']['activity_id']} - {'Fall' if result['activity_info']['is_fall'] else 'Non-Fall'}",
                        save_path=filter_plot_path
                    )
                    
                    # Plot predictions with moving average
                    if "raw_predictions" in result and len(result["raw_predictions"]) > 0:
                        preds_plot_path = os.path.join(self.output_dir, "plots", f"{result['activity_info']['filename']}_predictions_ma.png")
                        self.plot_predictions_with_ma(
                            result["raw_predictions"],
                            result["smoothed_predictions"],
                            title=f"Activity {result['activity_info']['activity_id']} - {'Fall' if result['activity_info']['is_fall'] else 'Non-Fall'}",
                            save_path=preds_plot_path
                        )
                
                # Collect results for metrics calculation
                activity_id = result["activity_info"]["activity_id"]
                is_fall = result["activity_info"]["is_fall"]
                
                true_labels.append(int(is_fall))
                pred_labels.append(int(result["is_fall_file"]))
                
                metrics_by_activity[activity_id].append({
                    "file_path": file_path,
                    "true_label": int(is_fall),
                    "pred_label": int(result["is_fall_file"]),
                    "correct": result["is_fall_file"] == is_fall
                })
                
                # Store only necessary data to avoid memory issues
                compact_result = {k: v for k, v in result.items() 
                               if k not in ["raw_predictions", "smoothed_predictions", "thresholded_predictions", 
                                          "window_results", "_raw_data", "_filtered_data"]}
                results.append(compact_result)
                
            except Exception as e:
                logger.error(f"Error in processing directory - file {file_path}: {e}")
                continue
        
        # Calculate overall metrics
        if not results:
            logger.warning("No results to analyze")
            return None
        
        if len(true_labels) > 0:
            # Calculate metrics
            accuracy = accuracy_score(true_labels, pred_labels)
            precision = precision_score(true_labels, pred_labels, zero_division=0)
            recall = recall_score(true_labels, pred_labels, zero_division=0)
            f1 = f1_score(true_labels, pred_labels, zero_division=0)
            
            # Calculate metrics by activity
            activity_metrics = {}
            for activity_id, activity_results in metrics_by_activity.items():
                if not activity_results:
                    continue
                
                act_true_labels = [r["true_label"] for r in activity_results]
                act_pred_labels = [r["pred_label"] for r in activity_results]
                
                # Skip activities with no predictions
                if not act_pred_labels:
                    continue
                
                # Calculate metrics for this activity
                act_accuracy = accuracy_score(act_true_labels, act_pred_labels)
                act_precision = precision_score(act_true_labels, act_pred_labels, zero_division=0)
                act_recall = recall_score(act_true_labels, act_pred_labels, zero_division=0)
                act_f1 = f1_score(act_true_labels, act_pred_labels, zero_division=0)
                
                # Store metrics
                activity_metrics[str(activity_id)] = {
                    "is_fall": activity_results[0]["true_label"] == 1,  # Assuming consistent labeling
                    "file_count": len(activity_results),
                    "correct_count": sum(r["correct"] for r in activity_results),
                    "accuracy": float(act_accuracy),
                    "precision": float(act_precision),
                    "recall": float(act_recall),
                    "f1_score": float(act_f1)
                }
            
            # Save metrics
            metrics = {
                "overall": {
                    "accuracy": float(accuracy),
                    "precision": float(precision),
                    "recall": float(recall),
                    "f1_score": float(f1),
                    "total_files": len(results)
                },
                "by_activity": activity_metrics
            }
            
            metrics_path = os.path.join(self.output_dir, "results", "metrics.json")
            with open(metrics_path, 'w') as f:
                json.dump(metrics, f, indent=2)
            
            # Print summary
            print("\n" + "="*50)
            print("DATASET INFERENCE SUMMARY (PEAK-ALIGNED WITH MOVING AVERAGE)")
            print("="*50)
            print(f"Total files processed: {len(results)}")
            print(f"Overall accuracy: {accuracy:.2%}")
            print(f"Precision: {precision:.2%}")
            print(f"Recall: {recall:.2%}")
            print(f"F1 Score: {f1:.2%}")
            print("="*50)
            
            # Create summary CSV
            summary_data = []
            for r in results:
                summary_data.append({
                    "filename": r["activity_info"]["filename"],
                    "subject_id": r["activity_info"]["subject_id"],
                    "activity_id": r["activity_info"]["activity_id"],
                    "is_fall_truth": r["activity_info"]["is_fall"],
                    "is_fall_pred": r["is_fall_file"],
                    "correct": r["correct_classification"],
                    "num_windows": r["num_windows"],
                    "fall_windows": r.get("fall_windows", 0),
                    "avg_prob": r["average_probability"]
                })
            
            summary_df = pd.DataFrame(summary_data)
            summary_path = os.path.join(self.output_dir, "results", "summary.csv")
            summary_df.to_csv(summary_path, index=False)
            
            return {
                "results": results,
                "metrics": metrics,
                "summary_df": summary_df
            }
        else:
            logger.warning("No valid predictions to calculate metrics")
            return {
                "results": results
            }

# Example usage
if __name__ == "__main__":
    import re
    
    # Configuration settings
    CONFIG = {
        "DATA_DIR": "data/smartfallmm/young/accelerometer/watch",
        "MODEL_PATH": "watchTousifKd.tflite",
        "OUTPUT_DIR": "peak_aligned_ma_results",
        "FILTER_CUTOFF": 7.5,
        "SAMPLING_RATE": 25.0,
        "FILTER_ORDER": 4,
        "WINDOW_SIZE": 128,  # Match model input shape
        "THRESHOLD": 0.50,   # Match MODEL_THRESHOLDS from Java code
        "FALL_PEAK_HEIGHT": 1.4,
        "FALL_PEAK_DISTANCE": 50,
        "NONFALL_PEAK_HEIGHT": 1.2,
        "NONFALL_PEAK_DISTANCE": 100,
        "MA_WINDOW_SIZE": 10,  # Size of moving average window
        "DECISION_THRESHOLD": 0.3,  # Threshold for file-level decision
        "MAX_PLOTS_PER_CATEGORY": 3,
    }
    
    # Create output directories
    os.makedirs(CONFIG["OUTPUT_DIR"], exist_ok=True)
    for dir_name in ["plots", "results", "raw_outputs"]:
        os.makedirs(os.path.join(CONFIG["OUTPUT_DIR"], dir_name), exist_ok=True)
    
    # Initialize processor and load model
    processor = PeakAlignedProcessor(CONFIG)
    model_loaded = processor.load_model(CONFIG["MODEL_PATH"])
    
    if model_loaded:
        # Process a sample file
        sample_files = glob.glob(os.path.join(CONFIG["DATA_DIR"], "*.csv"))
        if sample_files:
            sample_file = sample_files[0]
            print(f"Processing sample file: {sample_file}")
            result = processor.process_file(sample_file)
            
            # Process directory
            print(f"Processing all files in {CONFIG['DATA_DIR']}...")
            results = processor.process_directory()
            
            if results and "summary_df" in results:
                # Display summary statistics
                summary_df = results["summary_df"]
                print("\nSummary Statistics:")
                print(f"Total files: {len(summary_df)}")
                print(f"Fall files: {sum(summary_df['is_fall_truth'])}")
                print(f"Non-fall files: {sum(~summary_df['is_fall_truth'])}")
                print(f"Correctly classified: {sum(summary_df['correct'])}")
                print(f"Accuracy: {sum(summary_df['correct']) / len(summary_df):.2%}")
        else:
            print(f"No sample files found in {CONFIG['DATA_DIR']}")
    else:
        print("Failed to load model. Please check the path.")

2025-04-14 09:08:43,912 - PeakAlignedProcessor - INFO - Model loaded successfully from watchTousifKd.tflite
2025-04-14 09:08:43,913 - PeakAlignedProcessor - INFO - Input shape: [  1 128   4]
2025-04-14 09:08:43,913 - PeakAlignedProcessor - INFO - Input type: <class 'numpy.float32'>
2025-04-14 09:08:43,917 - PeakAlignedProcessor - ERROR - Error loading CSV file data/smartfallmm/young/accelerometer/watch/S33A08T08.csv: CSV must have at least 4 columns, found 1
2025-04-14 09:08:43,917 - PeakAlignedProcessor - ERROR - Error processing data/smartfallmm/young/accelerometer/watch/S33A08T08.csv: CSV must have at least 4 columns, found 1
2025-04-14 09:08:43,920 - PeakAlignedProcessor - INFO - Found 899 CSV files, processing...


Processing sample file: data/smartfallmm/young/accelerometer/watch/S33A08T08.csv
Processing all files in data/smartfallmm/young/accelerometer/watch...


Processing files:   0%|          | 0/899 [00:00<?, ?it/s]

2025-04-14 09:08:43,930 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A01T01.csv with shape (1109, 3)
2025-04-14 09:08:43,932 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A01T01.csv
2025-04-14 09:08:43,933 - PeakAlignedProcessor - INFO - Extracted 7 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S28A01T01.csv
2025-04-14 09:08:45,581 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A02T01.csv with shape (576, 3)
2025-04-14 09:08:45,583 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A02T01.csv
2025-04-14 09:08:45,584 - PeakAlignedProcessor - INFO - Extracted 4 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S28A02T01.csv
2025-04-14 09:08:47,267 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A02T02.csv with shape (3

2025-04-14 09:08:48,967 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A06T05.csv with shape (595, 3)
2025-04-14 09:08:48,970 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A06T05.csv
2025-04-14 09:08:48,971 - PeakAlignedProcessor - INFO - Extracted 6 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S28A06T05.csv
2025-04-14 09:08:48,975 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A07T02.csv with shape (425, 3)
2025-04-14 09:08:48,977 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S28A07T02.csv
2025-04-14 09:08:48,978 - PeakAlignedProcessor - INFO - Extracted 3 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S28A07T02.csv
2025-04-14 09:08:48,985 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S28A08T01.csv with shape (41

...

2025-04-14 09:08:58,988 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A03T03.csv with shape (2904, 3)
2025-04-14 09:08:58,990 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A03T03.csv
2025-04-14 09:08:58,992 - PeakAlignedProcessor - INFO - Extracted 22 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A03T03.csv
2025-04-14 09:08:59,005 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A03T04.csv with shape (2904, 3)
2025-04-14 09:08:59,007 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A03T04.csv
2025-04-14 09:08:59,008 - PeakAlignedProcessor - INFO - Extracted 22 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A03T04.csv
2025-04-14 09:08:59,022 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A03T05.csv with shape

2025-04-14 09:08:59,303 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A07T02.csv with shape (1366, 3)
2025-04-14 09:08:59,306 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A07T02.csv
2025-04-14 09:08:59,307 - PeakAlignedProcessor - INFO - Extracted 9 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A07T02.csv
2025-04-14 09:08:59,315 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A07T03.csv with shape (1366, 3)
2025-04-14 09:08:59,318 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A07T03.csv
2025-04-14 09:08:59,319 - PeakAlignedProcessor - INFO - Extracted 9 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A07T03.csv
2025-04-14 09:08:59,327 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A07T04.csv with shape (

2025-04-14 09:08:59,503 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A10T05.csv with shape (1176, 3)
2025-04-14 09:08:59,505 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A10T05.csv
2025-04-14 09:08:59,506 - PeakAlignedProcessor - INFO - Extracted 15 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A10T05.csv
2025-04-14 09:08:59,514 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A11T01.csv with shape (1586, 3)
2025-04-14 09:08:59,516 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A11T01.csv
2025-04-14 09:08:59,518 - PeakAlignedProcessor - INFO - Extracted 24 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A11T01.csv
2025-04-14 09:08:59,532 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A11T02.csv with shape

2025-04-14 09:08:59,775 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A14T04.csv with shape (557, 3)
2025-04-14 09:08:59,777 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A14T04.csv
2025-04-14 09:08:59,779 - PeakAlignedProcessor - INFO - Extracted 8 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A14T04.csv
2025-04-14 09:08:59,786 - PeakAlignedProcessor - INFO - Loaded data/smartfallmm/young/accelerometer/watch/S46A14T05.csv with shape (557, 3)
2025-04-14 09:08:59,787 - PeakAlignedProcessor - INFO - Applied Butterworth filter to data/smartfallmm/young/accelerometer/watch/S46A14T05.csv
2025-04-14 09:08:59,788 - PeakAlignedProcessor - INFO - Extracted 8 peak-aligned windows for data/smartfallmm/young/accelerometer/watch/S46A14T05.csv



DATASET INFERENCE SUMMARY (PEAK-ALIGNED WITH MOVING AVERAGE)
Total files processed: 830
Overall accuracy: 63.98%
Precision: 54.51%
Recall: 79.07%
F1 Score: 64.53%

Summary Statistics:
Total files: 830
Fall files: 344
Non-fall files: 486
Correctly classified: 531
Accuracy: 63.98%
